In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Makine Öğrenmesi Bootcamp/

/content/drive/MyDrive/Makine Öğrenmesi Bootcamp


In [3]:
%pwd

'/content/drive/MyDrive/Makine Öğrenmesi Bootcamp'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [6]:
df = pd.read_csv('fake_news_dataset.csv')

In [9]:
df.head(10)

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake
5,Method purpose mission approach professor short.,affect too bill whether kind project turn offi...,2023-12-02,Reuters,Sherri Fry,Health,real
6,Laugh member step.,often along newspaper establish fall president...,2022-06-01,CNN,Alyssa Young,Business,fake
7,Center measure difference dark.,ready movement bed increase during or history ...,2024-12-31,NY Times,Tina Garrett,Sports,fake
8,Moment make those affect first difference.,officer mention dream fill later foot suffer d...,2022-09-08,Fox News,Heather Greene,Entertainment,fake
9,Reason physical contain total decision.,choose anything treat beyond political minute ...,2022-11-08,Daily News,Erin Hanson,Health,real


In [11]:
df.info()
df['label'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     20000 non-null  object
 1   text      20000 non-null  object
 2   date      20000 non-null  object
 3   source    19000 non-null  object
 4   author    19000 non-null  object
 5   category  20000 non-null  object
 6   label     20000 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


,count
label,
fake,10056
real,9944


In [12]:
#author ve source da biner adet kayıp var
df['source'] = df['source'].fillna('Unknown')
df['author'] = df['author'].fillna('Unknown')


In [13]:
df.info()
#şimdi oldu

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     20000 non-null  object
 1   text      20000 non-null  object
 2   date      20000 non-null  object
 3   source    20000 non-null  object
 4   author    20000 non-null  object
 5   category  20000 non-null  object
 6   label     20000 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [16]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# İngilizce stopword'leri al
stop_words = set(stopwords.words('english'))

# 1. title + text birleştir
df['combined_text'] = df['title'] + " " + df['text']

# 2. Temizlik fonksiyonu
def clean_text(text):
    text = text.lower()  # küçük harfe çevir
    text = re.sub(r'\d+', '', text)  # sayıları çıkar
    text = text.translate(str.maketrans('', '', string.punctuation))  # noktalama işaretlerini çıkar
    text = ' '.join([word for word in text.split() if word not in stop_words])  # stopword temizliği
    return text

# 3. Temizlenmiş sütunu oluştur
df['clean_text'] = df['combined_text'].apply(clean_text)

# 4. Kontrol: İlk 3 satıra bakalım
df[['clean_text']].head(3)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,clean_text
0,foreign democrat final tax development store a...
1,offer resource great point probably guess west...
2,church carry identify forward present success ...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('clean_text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF vektörleyici tanımlanıyor
vectorizer = TfidfVectorizer(max_features=5000)  # En sık geçen 5000 kelime

# clean_text'e uygulama
X = vectorizer.fit_transform(df['clean_text']).toarray()

# Etiketleri sayıya çevir (Real: 0, Fake: 1)
y = df['label'].apply(lambda x: 1 if x.lower() == 'fake' else 0)


In [18]:
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (20000, 869)
y shape: (20000,)


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Veriyi ayır (eğitim %80, test %20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Modeli oluştur ve eğit
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 3. Tahmin yap
y_pred = model.predict(X_test)

# 4. Değerlendirme
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.50425

Confusion Matrix:
 [[ 978  993]
 [ 990 1039]]

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50      1971
           1       0.51      0.51      0.51      2029

    accuracy                           0.50      4000
   macro avg       0.50      0.50      0.50      4000
weighted avg       0.50      0.50      0.50      4000



In [20]:
df['label'].value_counts()


,count
label,
fake,10056
real,9944


In [21]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Modeli oluştur
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Eğit
rf_model.fit(X_train, y_train)

# 3. Tahmin yap
y_pred_rf = rf_model.predict(X_test)

# 4. Performans değerlendir
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.49525

Confusion Matrix:
 [[1058  913]
 [1106  923]]

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.54      0.51      1971
           1       0.50      0.45      0.48      2029

    accuracy                           0.50      4000
   macro avg       0.50      0.50      0.49      4000
weighted avg       0.50      0.50      0.49      4000



In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X = vectorizer.fit_transform(df['text'])


In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Veri Yükleme
df = pd.read_csv('fake_news_dataset.csv')

# 2. Label sütununu binary hale getir (eğer gerekirse)
df['label'] = df['label'].map({'fake': 1, 'real': 0})

# 3. TF-IDF vektörleştirme (unigram + bigram)
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X = vectorizer.fit_transform(df['text'])

# 4. Hedef değişken
y = df['label']

# 5. Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Model oluştur ve eğit (Logistic Regression)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 7. Test setinde tahmin yap
y_pred = model.predict(X_test)

# 8. Performans metriklerini yazdır
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.50575

Confusion Matrix:
 [[ 993  978]
 [ 999 1030]]

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50      1971
           1       0.51      0.51      0.51      2029

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000



In [26]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Model oluştur
svm_model = LinearSVC(max_iter=10000, random_state=42)

# Modeli eğit
svm_model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred_svm = svm_model.predict(X_test)

# Performans değerlendirme
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.50875

Confusion Matrix:
 [[ 995  976]
 [ 989 1040]]

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50      1971
           1       0.52      0.51      0.51      2029

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000



In [28]:
import pandas as pd

# Veri setini yükle
dg = pd.read_csv('fake_news_dataset.csv')

# İlk 5 satıra bakalım
print("Örnek veriler:\n", df.head())

# Metin uzunluklarına bakalım
dg['text_length'] = df['text'].apply(len)
print("\nMetin uzunluğu istatistikleri:")
print(dg['text_length'].describe())

# Çok kısa metin sayısı (örneğin 20 karakterden kısa olanlar)
short_texts = dg[dg['text_length'] < 20]
print(f"\n20 karakterden kısa metin sayısı: {len(short_texts)}")

# Duplicate kontrolü
duplicates = dg.duplicated(subset=['text'])
print(f"\nTekrarlanan (duplicate) metin sayısı: {duplicates.sum()}")

# Etiketlerin dağılımı
print("\nEtiket dağılımı:")
print(dg['label'].value_counts())


Örnek veriler:
                                   title  \
0               Foreign Democrat final.   
1   To offer down resource great point.   
2          Himself church myself carry.   
3                  You unit its should.   
4  Billion believe employee summer how.   

                                                text        date    source  \
0  more tax development both store agreement lawy...  2023-03-10  NY Times   
1  probably guess western behind likely next inve...  2022-05-25  Fox News   
2  them identify forward present success risk sev...  2022-09-01       CNN   
3  phone which item yard Republican safe where po...  2023-02-07   Reuters   
4  wonder myself fact difficult course forget exa...  2023-04-03       CNN   

                 author    category  label  
0          Paula George    Politics      0  
1           Joseph Hill    Politics      1  
2        Julia Robinson    Business      1  
3  Mr. David Foster DDS     Science      1  
4         Austin Walker  Techno